In [2]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [3]:
print(f'Length of dataset in characters: {len(text)}')
# Length of dataset in characters: 1115394

Length of dataset in characters: 1115394


In [4]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(f'vocabulary size: {vocab_size}')


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
vocabulary size: 65


In [6]:
# Tokenisation
# Create a mapping from chars to ints

s_to_i = { ch:i for i,ch in enumerate(chars) }
i_to_s = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [s_to_i[c] for c in s] # encoder: take a string, output a list of ints
decode = lambda l: ''.join([i_to_s[i] for i in l]) #decoder: take a list of integers, output a string


In [7]:
print(encode('hii there'))
print(decode(encode('hii there')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:300])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
BLOCK_SIZE = 8
print(train_data[:BLOCK_SIZE+1])
# there are 9 outputs, but this includes just 8 "examples"
# i.e. for tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
# we have that 18 -> 47, 18, 47 -> 56, 18, 47, 56 -> 57 and so on
# one symbol by itself does not constitute a training example

x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]
for t in range(BLOCK_SIZE):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target: {target}")

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
When input is tensor([18]) the target: 47
When input is tensor([18, 47]) the target: 56
When input is tensor([18, 47, 56]) the target: 57
When input is tensor([18, 47, 56, 57]) the target: 58
When input is tensor([18, 47, 56, 57, 58]) the target: 1
When input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
When input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
When input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [11]:
# Data loading
torch.manual_seed(1337)
BATCH_SIZE = 4 # num of indepenedent sequences to process in parallel
BLOCK_SIZE = 8 # length of each independent sequence - max context length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    index_list = torch.randint(len(data) - BLOCK_SIZE, (BATCH_SIZE,)) # 4 random indices from training set
    x = torch.stack([data[i:i+BLOCK_SIZE] for i in index_list])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1] for i in index_list])
    return x, y

xb, yb = get_batch('train')

print('inputs')
print(xb.shape)
print(xb)

print('targets')
print(yb.shape)
print(yb)

print('------')

for b in range(BATCH_SIZE): # batch dim
    for t in range(BLOCK_SIZE): # time dim
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target: {target}')


inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
------
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

In [12]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

# Bigram Language Model: simplest language model
# character-level language model that treats each character as a token and 
# uses it to predict the next character.
# It deals with only 2 characters at a time, one which is passed and the 
# other which it has to predict. 
# It cannot take all the previous characters for prediction so it has a 
# limited range of context.
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx and targets are both BxT tensor of integers
        logits = self.token_embedding_table(idx) #BxTxC

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context

        for _ in range(max_new_tokens):
            # get preds
            logits, loss = self(idx) # = self.forward(idx)

            # focus only on last timestep
            logits = logits[:, -1, :] # becomes (B, C)

            # apply softmax
            probs = F.softmax(logits, dim=-1) # (B, C)

            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)

            # append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        
        return idx

bigramModel = BigramLanguageModel(vocab_size)
logits, loss = bigramModel(xb, yb)
print(logits.shape)
print(loss)

# we have 65 possible vocab elements (vocab_size)
# expected loss = -ln(1/65) = 4.17
# Result: 
# torch.Size([32, 65])
# tensor(4.8786, grad_fn=<NllLossBackward0>)

idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(bigramModel.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [13]:
# create optimiser
optimizer = torch.optim.AdamW(bigramModel.parameters(), lr=1e-3)

In [14]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # loss and backprop
    logits, loss = bigramModel(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4212486743927


In [15]:
idx = torch.zeros((1, 1), dtype=torch.long)
print(decode(bigramModel.generate(idx, max_new_tokens=500)[0].tolist()))


KIZ!

we?
W$d he k
MIV;


IETyoZY st:


Gheay Yatondgrende'TEQUNathinin: ans nd s

OREre.
Qhour im, sof bn I's temrpe thithattersuravWhefe h antr forgg m walos nt kit mo RLORZ'd IFknyor!
Sindis prewhous oodod.
wouprebe bll:CUNTHUSt

Thleaix;
Malesheais whe; lino.
ALI;
CXdaimboth,
batly, tyous w F nig har-
ASerin livin ad:
GRveitho wond ve ack'
3'd gat held inghe beromUCOFFoce ?Y's.
Pat fe, healcofoan.
FXgrnk3ffstoncetasollar dir ce s:

FGS OXELEurdo the horer mf IOP,-W?ENe,
LIDbor d,

CUSY: CKr 


# Mathematical Trick in Self-attention

In [16]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [17]:
# Version 1: We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)

xbow[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [18]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [19]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [20]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf')) # this is only in decoder blocks
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [21]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [22]:
print(k.var())
print(q.var())
print(wei.var())

tensor(1.0918)